In [ ]:
import pandas as pd
import numpy as np

def main():
    print("🎬 Hybrid Movie Recommendation System")
    
    # Load or download data
    ratings, movies = download_movielens()
    if ratings is None or movies is None:
        print("Failed to load data. Exiting.")
        return
    ratings, movies, stats = preprocess_data(ratings, movies)
    if ratings is None or movies is None:
        print("Failed to preprocess data. Exiting.")
        return
    cosine_sim = setup_content_based(movies)
    svd, predictions, testset = setup_collaborative(ratings)
    if cosine_sim is None or svd is None:
        print("Failed to set up models. Exiting.")
        return
    metrics = evaluate_model(ratings, predictions, testset)

    while True:
        print("\nChoose mode:")
        print("1. Home")
        print("2. Content-Based")
        print("3. Collaborative")
        print("4. Hybrid")
        print("5. Evaluation")
        print("6. Exit")
        app_mode = input("Enter choice (1-6): ")

        if app_mode == "1":
            print("\n## MovieLens 100K Dataset Overview")
            print(f"Total Movies: {stats['movie_count']}")
            print(f"Total Ratings: {stats['rating_count']}")
            print(f"Total Users: {stats['user_count']}")
            print("Sample Movies:")
            print(movies[['movieId', 'title', 'genres']].head(5).to_string())

        elif app_mode == "2":
            print("\nContent-Based Recommendations")
            movie = input("Enter movie title: ")
            try:
                n_recs = int(input("Number of recommendations (1-20): "))
                if 1 <= n_recs <= 20:
                    recs = get_content_recommendations(movie, movies, cosine_sim, n_recs)
                    if not recs.empty:
                        print(recs.to_string())
                    else:
                        print("No recommendations available.")
                else:
                    print("Please enter a number between 1 and 20.")
            except ValueError:
                print("Invalid input. Please enter a number.")

        elif app_mode == "3":
            print("\nCollaborative Filtering Recommendations")
            try:
                user_id = int(input("Enter User ID: "))
                n_recs = int(input("Number of recommendations (1-20): "))
                if 1 <= n_recs <= 20:
                    recs = get_collab_recommendations(user_id, ratings, movies, svd, n_recs)
                    if not recs.empty:
                        print(recs.to_string())
                    else:
                        print("No recommendations available.")
                else:
                    print("Please enter a number between 1 and 20.")
            except ValueError:
                print("Invalid input. Please enter a number.")

        elif app_mode == "4":
            print("\nHybrid Recommendations")
            try:
                user_id = int(input("Enter User ID: "))
                movie = input("Enter movie title: ")
                content_w = float(input("Content-Based Weight (0.0-1.0): "))
                collab_w = float(input("Collaborative Weight (0.0-1.0): "))
                n_recs = int(input("Number of recommendations (1-20): "))
                if 0 <= content_w <= 1 and 0 <= collab_w <= 1 and 1 <= n_recs <= 20:
                    recs = hybrid_recommendations(user_id, movie, ratings, movies, svd, cosine_sim, n_recs, content_w, collab_w)
                    if not recs.empty:
                        print(recs.to_string())
                    else:
                        print("No recommendations available.")
                else:
                    print("Weights must be between 0.0 and 1.0, and recommendations between 1 and 20.")
            except ValueError:
                print("Invalid input. Please enter valid numbers.")

        elif app_mode == "5":
            print("\nModel Performance")
            print(f"RMSE: {metrics['rmse']:.4f}")
            print(f"MAE: {metrics['mae']:.4f}")
            print(f"Precision@10: {metrics['precision']:.4f}")
            print(f"Recall@10: {metrics['recall']:.4f}")
            print(f"F1 Score: {metrics['f1']:.4f}")

        elif app_mode == "6":
            print("Exiting.")
            break

        else:
            print("Invalid choice. Please enter a number between 1 and 6.")

if __name__ == "__main__":
    main()

# Note: download_movielens, preprocess_data, setup_content_based, setup_collaborative, get_content_recommendations, 
# get_collab_recommendations, hybrid_recommendations, and evaluate_model are assumed to be imported or defined elsewhere